[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/MASt3R-jupyter/blob/main/MASt3R_jupyter.ipynb)

In [ ]:
%cd /content
!git clone -b dev --recursive https://github.com/camenduru/mast3r
%cd /content/mast3r

%cd dust3r/croco/models/curope
!python setup.py build_ext --inplace

!mkdir -p /content/mast3r/checkpoints
!wget https://huggingface.co/camenduru/mast3r/resolve/main/MASt3R_ViTLarge_BaseDecoder_512_catmlpdpt_metric.pth -O /content/mast3r/checkpoints/MASt3R_ViTLarge_BaseDecoder_512_catmlpdpt_metric.pth

!pip install roma einops trimesh
!pip install /content/mast3r/dust3r/croco/models/curope

!wget https://raw.githubusercontent.com/naver/mast3r/refs/heads/main/assets/NLE_tower/01D90321-69C8-439F-B0B0-E87E7634741C-83120-000041DAE419D7AE.jpg -O /content/1.jpg
!wget https://raw.githubusercontent.com/naver/mast3r/refs/heads/main/assets/NLE_tower/1AD85EF5-B651-4291-A5C0-7BDB7D966384-83120-000041DADF639E09.jpg -O /content/2.jpg
!wget https://raw.githubusercontent.com/naver/mast3r/refs/heads/main/assets/NLE_tower/2679C386-1DC0-4443-81B5-93D7EDE4AB37-83120-000041DADB2EA917.jpg -O /content/3.jpg
!wget https://raw.githubusercontent.com/naver/mast3r/refs/heads/main/assets/NLE_tower/28EDBB63-B9F9-42FB-AC86-4852A33ED71B-83120-000041DAF22407A1.jpg -O /content/4.jpg
!wget https://raw.githubusercontent.com/naver/mast3r/refs/heads/main/assets/NLE_tower/91E9B685-7A7D-42D7-B933-23A800EE4129-83120-000041DAE12C8176.jpg -O /content/5.jpg
!wget https://raw.githubusercontent.com/naver/mast3r/refs/heads/main/assets/NLE_tower/CDBBD885-54C3-4EB4-9181-226059A60EE0-83120-000041DAE0C3D612.jpg -O /content/6.jpg
!wget https://raw.githubusercontent.com/naver/mast3r/refs/heads/main/assets/NLE_tower/FF5599FD-768B-431A-AB83-BDA5FB44CB9D-83120-000041DADDE35483.jpg -O /content/7.jpg

In [ ]:
import os
import torch
import math

from mast3r.demo import get_reconstructed_scene
from mast3r.model import AsymmetricMASt3R
from mast3r.utils.misc import hash_md5
from dust3r.demo import set_print_with_timestamp

# for gpu >= Ampere and pytorch >= 1.12
torch.backends.cuda.matmul.allow_tf32 = True
batch_size = 1
set_print_with_timestamp()

weights_path = "/content/mast3r/checkpoints"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = AsymmetricMASt3R.from_pretrained(weights_path).to(device)
chkpt_tag = hash_md5(weights_path)

# Set the temporary directory
temp_dir = '/content/temp'
os.makedirs(temp_dir, exist_ok=True)  # Ensure the temp directory exists

image_size = 512
silent = True
gradio_delete_cache = 7200

class FileState:
    def __init__(self, outfile_name=None):
        self.outfile_name = outfile_name

    def __del__(self):
        if self.outfile_name is not None and os.path.isfile(self.outfile_name):
            os.remove(self.outfile_name)
        self.outfile_name = None

def local_get_reconstructed_scene(filelist, min_conf_thr, matching_conf_thr,
                                  as_pointcloud, cam_size,
                                  shared_intrinsics, **kw):
    lr1 = 0.07
    niter1 = 500
    lr2 = 0.014
    niter2 = 200
    optim_level = 'refine'
    mask_sky, clean_depth, transparent_cams = False, True, False
    
    if len(filelist) < 5:
        scenegraph_type = 'complete'
        winsize = 1
    else:
        scenegraph_type = 'logwin'
        half_size = math.ceil((len(filelist) - 1) / 2)
        max_winsize = max(1, math.ceil(math.log(half_size, 2)))
        winsize = min(5, max_winsize)
        
    refid = 0
    win_cyclic = False
    scene_state, outfile = get_reconstructed_scene(temp_dir, gradio_delete_cache, model, device, silent, image_size, None,
                                                   filelist, optim_level, lr1, niter1, lr2, niter2, min_conf_thr, matching_conf_thr,
                                                   as_pointcloud, mask_sky, clean_depth, transparent_cams, cam_size, scenegraph_type, winsize,
                                                   win_cyclic, refid, TSDF_thresh=0, shared_intrinsics=shared_intrinsics, **kw)
    
    filestate = FileState(scene_state.outfile_name)
    scene_state.outfile_name = None
    del scene_state
    return filestate, outfile

def run_example(snapshot, matching_conf_thr, min_conf_thr, cam_size, as_pointcloud, shared_intrinsics, filelist, **kw):
    return local_get_reconstructed_scene(filelist, min_conf_thr, matching_conf_thr, as_pointcloud, cam_size, shared_intrinsics, **kw)

filelist = [
    "/content/1.jpg",
    "/content/2.jpg",
    "/content/3.jpg",
    "/content/4.jpg",
    "/content/5.jpg",
    "/content/6.jpg",
    "/content/7.jpg"
]
matching_conf_thr = 2.0
min_conf_thr = 1.5
cam_size = 0.2
as_pointcloud = False
shared_intrinsics = True

filestate, outfile = run_example(None, matching_conf_thr, min_conf_thr, cam_size, as_pointcloud, shared_intrinsics, filelist)
print(f"Reconstructed scene saved to: {outfile}")
!mv {outfile} /content/scene.glb